In [1]:
import pymongo
import pprint
from pymongo import MongoClient
print(pymongo.__version__)

4.3.3


https://pymongo.readthedocs.io/en/stable/tutorial.html

In [2]:
client = MongoClient(host='localhost', port=27017)
# or use uri format
# client = MongoClient('mongodb://localhost:27017/')
try:
    print(client.server_info())
except Exception:
    print("Unable to connect to the server.")
# show current database names
print(client.list_database_names())

{'version': '6.0.4', 'gitVersion': '44ff59461c1353638a71e710f385a566bcd2f547', 'modules': [], 'allocator': 'tcmalloc', 'javascriptEngine': 'mozjs', 'sysInfo': 'deprecated', 'versionArray': [6, 0, 4, 0], 'openssl': {'running': 'OpenSSL 3.0.2 15 Mar 2022', 'compiled': 'OpenSSL 3.0.2 15 Mar 2022'}, 'buildEnvironment': {'distmod': 'ubuntu2204', 'distarch': 'x86_64', 'cc': '/opt/mongodbtoolchain/v3/bin/gcc: gcc (GCC) 8.5.0', 'ccflags': '-Werror -include mongo/platform/basic.h -ffp-contract=off -fasynchronous-unwind-tables -ggdb -Wall -Wsign-compare -Wno-unknown-pragmas -Winvalid-pch -fno-omit-frame-pointer -fno-strict-aliasing -O2 -march=sandybridge -mtune=generic -mprefer-vector-width=128 -Wno-unused-local-typedefs -Wno-unused-function -Wno-deprecated-declarations -Wno-unused-const-variable -Wno-unused-but-set-variable -Wno-missing-braces -fstack-protector-strong -fdebug-types-section -Wa,--nocompress-debug-sections -fno-builtin-memcmp', 'cxx': '/opt/mongodbtoolchain/v3/bin/g++: g++ (GCC) 

In [4]:
# Select database
db = client['new_db']  # or `db = client.new_db`

# Show current collection names
print('Collection names:', db.list_collection_names())

# Insert a document into the collection
db.test.insert_one({'name': 'test', 'age': 20})

# Check the collection names again
print('Collection names:', db.list_collection_names()) 

# Drop collections
db.test.drop()

# Check the collection names again
print('Collection names:', db.list_collection_names())

Collection names: []
Collection names: ['test']
Collection names: []


In [5]:
# Let's insert a few more documents
db.test.insert_many([
    {'name': 'joe', 'age': 21},
    {'name': 'sam', 'age': 20},
    {'name': 'john', 'age': 19},
    {'name': 'amy', 'age': 23}
])

# Filter the documents by age > 20 and project the name field
for res in db.test.find(filter={'age': {'$gt': 20}}, projection={'_id': 0, 'name': 1}):
    pprint.pprint(res)

{'name': 'joe'}
{'name': 'amy'}


# Sample Analytics Dataset 

- https://www.mongodb.com/docs/atlas/sample-data/sample-analytics/
- https://github.com/mcampo2/mongodb-sample-databases/tree/master/sample_analytics

| Collection Name | Description
|---|---
| accounts | Contains details on customer accounts.
| customers | Contains details on customers.
| transactions | Contains customer transactions.

A single customer can have multiple accounts, transaction refer `account_id`

## Schema Information with example(BSON)

### `accounts` sample data

``` json
{
    "_id": {"$oid": "5ca4bbc7a2dd94ee5816238c" },  // ObjectID
    "account_id": { "$numberInt": "371138" },  // Integer
    "limit": { "$numberInt": "9000" },  // Integer
    "products": [ "Derivatives", "InvestmentStock"]  // Array
}
```

### `customers` sample data

``` json
{
    "_id": { "$oid": "5ca4bbcea2dd94ee58162a68" },  // ObjectID
    "username": "fmiller",  // String
    "name": "Elizabeth Ray",  // String
    "address": "9286 Bethany Glens\nVasqueztown, CO 22939",  // String
    "birthdate":{ "$date": { "$numberLong": "226117231000" } },  // Embedded Document
    "email": "arroyocolton@gmail.com",  // String
    "active": true,  // Boolean
    "accounts":[
        {"$numberInt":"371138"}, {"$numberInt":"324287"}, {"$numberInt":"276528"}, 
        {"$numberInt":"332179"}, {"$numberInt":"422649"}, {"$numberInt":"387979"}
    ], // Array
    "tier_and_details": {
        "0df078f33aa74a2e9696e0520c1a828a": { 
            "tier":"Bronze", 
            "id":"0df078f33aa74a2e9696e0520c1a828a",
            "active":true,
            "benefits": ["sports tickets"]
        },
        "699456451cc24f028d2aa99d7534c219": {
            "tier":"Bronze",
            "benefits": ["24 hour dedicated line", "concierge services"],
            "active":true, "id":"699456451cc24f028d2aa99d7534c219"
        }
    }
}
```

### `transactions` sample data

``` json
{
    "_id": { "$oid": "5ca4bbc1a2dd94ee58161cb1" },  // ObjectID
    "account_id": { "$numberInt": "443178" },  // Integer
    "transaction_count": { "$numberInt": "66" },  // Integer
    "bucket_start_date": { "$date": { "$numberLong": "-28598400000" } },  // Date
    "bucket_end_date": { "$date": { "$numberLong": "1483401600000" } },  // Date
    "transactions": [
        {
            "date": { "$date": { "$numberLong": "1063065600000" } },  // Date
            "amount": { "$numberInt": "7514" },  // Integer
            "transaction_code": "buy",  // String
            "symbol": "adbe",  // String
            "price": "19.1072802650074180519368383102118968963623046875",  // Float
            "total": "143572.1039112657392422534031"  // Float
        },  
        // ...
        {
            "date": { "$date": { "$numberLong": "1120694400000" } },
            "amount": { "$numberInt": "2881" },
            "transaction_code":"buy",
            "symbol": "msft",
            "price": "20.6769287918292690164889791049063205718994140625",
            "total": "59570.23184926012403650474880"
        }
    ]
}

```

In [7]:
from pathlib import Path
import json
from bson import json_util

data_path = Path('./datasets/sample_analytics/')
db = client['analytics']  # select database

In [8]:
def jsonl_to_bson(path):
    with open(path) as file:
        data = [json.loads(x, object_hook=json_util.object_hook) for x in file.readlines()]
    return data

for file_name in ['accounts.json', 'customers.json', 'transactions.json']:
    collection_name = file_name.split('.')[0]
    collection = db[collection_name]  # select collection
    collection.insert_many(jsonl_to_bson(data_path / file_name))

In [26]:
# print out the collection names
pprint.pprint(db.list_collection_names())

# search user 'fmiller' in the accounts collection
res = db.customers.find(
    filter={'username': 'fmiller'}, 
    projection={'_id': 0, 'username': 1, 'active': 1, 'name': 1, 'accounts': 1}
)
for doc in res:
    pprint.pprint(doc)

['customers', 'accounts', 'transactions']
{'accounts': [371138, 324287, 276528, 332179, 422649, 387979],
 'active': True,
 'name': 'Elizabeth Ray',
 'username': 'fmiller'}


* Lookup: https://www.mongodb.com/docs/manual/reference/operator/aggregation/lookup/
* Reduce: https://www.mongodb.com/docs/manual/reference/operator/aggregation/reduce/
* Set Union: https://www.mongodb.com/docs/manual/reference/operator/aggregation/setUnion/

In [77]:
res = db.customers.aggregate([
    {'$match': {'username': 'fmiller'}},
    {'$project': {'_id': 0, 'name': 1, 'accounts': 1}},
    {'$lookup': {
        'from': 'transactions',
        'localField': 'accounts',
        'foreignField': 'account_id',
        'as': 'transactions',
        '$pipeline': [
            {'$project': {'_id': 0, 'account_id': 1, 'amount': 1, 'date': 1}}
        ]
    }},
    {'$project': {'products_type': '$products_type.products', 'name': 1}},
    {'$addFields': {
        'products_type': {
            '$reduce': {
                'input': '$products_type',
                'initialValue': [],
                'in': {'$setUnion': ['$$value', '$$this']}  # $$this is the current element in the input array
            }
        }
    }},
])

for doc in res:
    pprint.pprint(doc)

{'name': 'Elizabeth Ray',
 'products_type': ['Brokerage',
                   'Commodity',
                   'CurrencyService',
                   'Derivatives',
                   'InvestmentFund',
                   'InvestmentStock']}


In [81]:
fmiller_accounts = db.customers.find_one({"username": "fmiller"})["accounts"]
pipeline = [
    {"$match": {"account_id": {"$in": fmiller_accounts}, "transaction_code": {"$in": ["buy", "sell"]}}},
    {"$group": {"_id": "$transaction_code", "total": {"$sum": "$total"}}},
    {"$project": {"_id": 0, "buy_total": {"$cond": [{"$eq": ["$_id", "buy"]}, "$total", 0]}, 
                             "sell_total": {"$cond": [{"$eq": ["$_id", "sell"]}, "$total", 0]}}},
    {"$group": {"_id": None, "buy_total": {"$sum": "$buy_total"}, "sell_total": {"$sum": "$sell_total"}}},
    {"$project": {"_id": 0, "profit": {"$subtract": ["$buy_total", "$sell_total"]}}}
]
result = db.transactions.aggregate(pipeline)
for doc in result:
    pprint.pprint(doc)

- concatArrays: https://www.mongodb.com/docs/manual/reference/operator/aggregation/concatArrays/

In [152]:
res = db.customers.aggregate([
    {'$match': {'username': 'fmiller'}},
    {'$project': {'_id': 0, 'accounts': 1}},
    {'$lookup': {
        'from': 'transactions',
        'localField': 'accounts',
        'foreignField': 'account_id',
        'as': 'transactions_info',
    }},
    {'$unwind': '$transactions_info'},
    {'$project': {
        'transactions': '$transactions_info.transactions'
    }},
    {'$unwind': '$transactions'},
    {'$project': {
        'symbol': '$transactions.symbol',
        'total': {'$cond': [
            {'$eq': ['$transactions.transaction_code', 'buy']}, 
            {'$multiply': [{'$toDouble': '$transactions.total'}, -1]}, 
            {'$toDouble': '$transactions.total'}
            ]
        },
    }},
    {'$group': {
        '_id': '$symbol',
        'total': {'$sum': '$total'}
    }},
])

for doc in res:
    pprint.pprint(doc)

{'_id': 'intc', 'total': -542047.6980640853}
{'_id': 'crm', 'total': 186904.20184876822}
{'_id': 'csco', 'total': 688125.9080473727}
{'_id': 'nvda', 'total': 834548.3059912496}
{'_id': 'amd', 'total': 16521.06362185528}


In [164]:
# check if fmiller has those symbols
account_list = db.customers.find({'username': 'fmiller'})[0]['accounts']
db.transactions.distinct('transactions.symbol', {'account_id': {'$in': account_list}})

['amd', 'crm', 'csco', 'intc', 'nvda']